In [9]:
import pandas as pd
words=[]
counts=[]
with open("hw2_word_counts_05.txt") as f:
    for i in f.readlines():
        i=i.strip("\n").split(" ")
        words.append(i[0])
        counts.append(i[1])
df=pd.DataFrame({'word': words, 'count': counts})
        


In [18]:
df.shape
df['count']=df['count'].astype(int)

In [17]:
import numpy as np

In [20]:
count_total=sum(df['count'])
df['prior probability']=df['count'].apply(lambda x: x/count_total)

## (a)

In [37]:
# the 10 least frequent 5-letter words
df_sort=df.sort_values(by=['prior probability'], ascending=True)
print(df_sort.head(10)['word'].tolist())

['MAPCO', 'BOSAK', 'CAIXA', 'OTTIS', 'TROUP', 'CLEFT', 'FOAMY', 'CCAIR', 'SERNA', 'YALOM']


In [38]:
# the 10 most frequent 5-letter words
df_sort=df.sort_values(by=['prior probability'], ascending=False)
print(df_sort.head(10)['word'].tolist())

['THREE', 'SEVEN', 'EIGHT', 'WOULD', 'ABOUT', 'THEIR', 'WHICH', 'AFTER', 'FIRST', 'FIFTY']


## (b) Consider the following stages of the game.  For each of the following, indicate the best next guess—namely, the letter`that is most likely (probable) to be among the missing letters.  Also report theprobabilityP(Li=`for somei∈{1,2,3,4,5}|E)for your guess`.  Your answers should fill in thelast two columns of this table. 

In [169]:
import string
all_letters=list(string.ascii_uppercase)
len(all_letters)

26

In [409]:
def best_guess(positions, condition):
    probs_letter=[]
    selected_df=translate_condition(condition)
    incorrect_guess=[list(x)[1:] for x in condition if x[0]=='!'][0]
    existed_letters=[ x for x in condition if x[0]!="!"]
    combined_letters=incorrect_guess+existed_letters
    available_letter=np.array([x for x  in all_letters if x not in combined_letters ])
    for letter in available_letter:
        probs=0
        for w in words:
            prob=letter_given_word(letter, positions, w)*word_given_condition(w, selected_df)
            probs+=prob
        probs_letter.append(probs)
    dic=dict(zip(available_letter, probs_letter))
    sorted_dic = sorted(dic.items(), key=lambda x: x[1])
    return sorted_dic

In [410]:
#input "condition" looks like ['A','!DIM', 'A','!DIM','A']
def translate_condition(condition):
    bool_index=np.array([0]*len(words))
    for i in range(5):
        if condition[i][0]=='!':
            index=np.array([1 if w[i] not in list(condition[i][1:]) else 0 for w in words])
        else:
            index=np.array([1 if w[i] in list(condition[i]) else 0 for w in words])
        bool_index+=index
    boolean=np.array([True if x==5 else False for x in bool_index])
    selected_df=df.loc[boolean]

    return selected_df
        
        
    
    

In [411]:
#P(L2=letter or L4=letter|W=word)
def letter_given_word(letter, positions, w):
    all_letters=[w[p] for p in positions]
    if letter in all_letters:
        return 1
    else:
        return 0


In [412]:
#P(W=word|['A','!DIM', 'A','!DIM','A'])
def word_given_condition(word, selected_df):
    selected_words=selected_df['word'].tolist()
    if word in selected_words:
        prob=(selected_df.loc[df['word']==word, 'prior probability'].tolist()[0])/sum_over_words(selected_df)
        return prob
    else:
        return 0

In [413]:
#sum over all W=word P(W=word)*P(['A','!DIM', 'A','!DIM','A']|W=word)
def sum_over_words(selected_df):
    denominator=sum(selected_df['prior probability'].tolist())
    return denominator

In [414]:
best_guess(positions=range(5), condition=["!", "!", '!', "!", "!"])
#E

[('Q', 0.00427105163214392),
 ('Z', 0.004292056590227283),
 ('J', 0.0156305590567442),
 ('X', 0.02074950648133407),
 ('K', 0.05958258581993068),
 ('V', 0.0636935561876753),
 ('P', 0.07724932115497987),
 ('B', 0.08609971458045383),
 ('M', 0.09001668263347835),
 ('F', 0.10400220121523383),
 ('Y', 0.10544123654231163),
 ('W', 0.10968606459324604),
 ('G', 0.11007759179329685),
 ('C', 0.1490947580626736),
 ('U', 0.1563097654659429),
 ('D', 0.17146229864431783),
 ('L', 0.20103153914025848),
 ('N', 0.23201711395267793),
 ('H', 0.2597325690485792),
 ('I', 0.30255306785240443),
 ('O', 0.3146699279582096),
 ('A', 0.3468993615927841),
 ('S', 0.3498943294049684),
 ('R', 0.38310199394456784),
 ('T', 0.4306932275448822),
 ('E', 0.5394172389647948)]

In [415]:
best_guess(positions=range(5), condition=["!EO", "!EO", '!EO', "!EO", "!EO"])
#I

[('Z', 0.004600009630695309),
 ('Q', 0.008808839896183291),
 ('J', 0.011516255585668045),
 ('V', 0.028985687596476108),
 ('X', 0.043630296003207454),
 ('B', 0.07800646771525846),
 ('K', 0.0903743364694415),
 ('M', 0.10066457208061465),
 ('W', 0.10232343229323658),
 ('P', 0.10274977992779173),
 ('G', 0.11908030106202792),
 ('D', 0.1265246121075158),
 ('U', 0.14430027858679872),
 ('Y', 0.18198713707022407),
 ('F', 0.18460744366178572),
 ('C', 0.20778549736319193),
 ('L', 0.24638456664846997),
 ('N', 0.2588422497520644),
 ('H', 0.2725356917355115),
 ('R', 0.3098751959278397),
 ('S', 0.42672258241142985),
 ('A', 0.44255209097003284),
 ('T', 0.485750087785412),
 ('I', 0.6365554141009618)]

In [416]:
best_guess(positions=range(1, 5), condition=["Q", "!", '!', "!", "!"])
#U

[('F', 0.0),
 ('J', 0.0),
 ('M', 0.0),
 ('V', 0.0),
 ('W', 0.0),
 ('X', 0.0),
 ('Z', 0.0),
 ('B', 0.0011342369420972046),
 ('G', 0.0019282028015652477),
 ('D', 0.0024386094255089896),
 ('P', 0.003289287132081893),
 ('H', 0.005501049169171442),
 ('Y', 0.005501049169171442),
 ('R', 0.029093177564793293),
 ('L', 0.03958486927919244),
 ('S', 0.07202404582317248),
 ('O', 0.07996370441785292),
 ('N', 0.10905688198264621),
 ('C', 0.17132649010378273),
 ('A', 0.2078489196393127),
 ('K', 0.2873022174332219),
 ('T', 0.5447740032892873),
 ('I', 0.6393693642601942),
 ('E', 0.6668746101060513),
 ('U', 0.9866727159303582)]

In [417]:
best_guess(positions=range(1, 5), condition=["Q", "!U", '!U', "!U", "!U"])
#A

[('C', 0.0),
 ('D', 0.0),
 ('E', 0.0),
 ('F', 0.0),
 ('G', 0.0),
 ('H', 0.0),
 ('I', 0.0),
 ('J', 0.0),
 ('K', 0.0),
 ('L', 0.0),
 ('M', 0.0),
 ('N', 0.0),
 ('O', 0.0),
 ('P', 0.0),
 ('V', 0.0),
 ('W', 0.0),
 ('X', 0.0),
 ('Y', 0.0),
 ('Z', 0.0),
 ('B', 0.0851063829787234),
 ('S', 0.0851063829787234),
 ('R', 0.9148936170212765),
 ('T', 0.9148936170212765),
 ('A', 0.9999999999999999)]

In [419]:
best_guess(positions=[0,1,4], condition=["!ADIR", "!ADIR", 'Z', "E", "!ADIR"])
#U

[('C', 0.0),
 ('G', 0.0),
 ('H', 0.0),
 ('L', 0.0),
 ('M', 0.0),
 ('P', 0.0),
 ('Q', 0.0),
 ('S', 0.0),
 ('T', 0.0),
 ('V', 0.0),
 ('W', 0.0),
 ('X', 0.0),
 ('Y', 0.0),
 ('N', 0.11965811965811965),
 ('U', 0.11965811965811965),
 ('B', 0.21367521367521367),
 ('K', 0.21367521367521367),
 ('F', 0.6666666666666666),
 ('J', 0.6666666666666666),
 ('O', 0.8803418803418803)]